# import lib

In [2]:
!pip install feature-engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.1/357.1 kB 19.9 MB/s eta 0:00:00


In [1]:
!pip install xgboost


  Using cached xgboost-2.1.0.tar.gz (1.1 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 2.2 MB/s eta 0:00:00:00:0100:01
  Created wheel for xgboost: filename=xgboost-2.1.0-py3-none-linux_x86_64.whl size=4190309 sha256=3215523a00957c8d9fafc6139c95c4969a23c1536695a0fbc5198612df260458
  Stored in directory: /home/ec2-user/.cache/pip/wheels/8f/9c/2a/00927b25214c3cb88704f768fd0fe6df99c701e3286023ffd9
Successfully built xgboost


In [70]:
conda install -c conda-forge gcc_linux-64


Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::autopep8==2.0.4=pyhd8ed1ab_0
  - conda-forge/linux-64::black==24.3.0=py310hff52083_0
  - conda-forge/noarch::bleach==6.1.0=pyhd8ed1ab_0
  - conda-forge/noarch::plotly==5.19.0=pyhd8ed1ab_0
  - conda-forge/noarch::pytest==8.1.1=pyhd8ed1ab_0
  - conda-forge/noarch::qtpy==2.4.1=pyhd8ed1ab_0
  - conda-forge/linux-64::sip==6.7.12=py310hc6cd4ac_0
  - conda-forge/noarch::flask==3.0.2=pyhd8ed1ab_0
  - conda-forge/noarch::importlib_metadata==7.1.0=hd8ed1ab_0
  - conda-forge/linux-64::pyqt5-sip==12.12.2=py310hc6cd4ac_5
  - conda-forge/noarch::pytoolconfig==1.2.5=pyhd8ed1ab_0
  - conda-forge/noarch::qdarkstyle==3.1=pyhd8ed1ab_0
  - conda-forge/noarch::qtawesome==1.3.0=pyh9208f05_0
  - conda-forge/noarch::yapf==0.40.1=pyhd8ed1ab_0
  - conda-forge/noarch::dask-core==2024.3.1=pyhd8ed1ab_0
  - conda-forge/noarch::flask-cors==4.0

In [72]:
conda install gxx_linux-64


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.5.0



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs:
    - gxx_linux-64


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    gxx_impl_linux-64-13.2.0   |      h2a599c4_12        12.6 MB  conda-forge
    gxx_linux-64-13.2.0        |       h4ebd915_9          29 KB  conda-forge
    libstdcxx-devel_linux-64-13.2.0|     hdb50d1a_112        12.5 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        25.1 MB

The following NEW packages will be INSTAL

In [68]:
conda install -c anaconda gcc_linux-64


Retrieving notices: ...working... done
Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::autopep8==2.0.4=pyhd8ed1ab_0
  - conda-forge/linux-64::black==24.3.0=py310hff52083_0
  - conda-forge/noarch::bleach==6.1.0=pyhd8ed1ab_0
  - conda-forge/noarch::plotly==5.19.0=pyhd8ed1ab_0
  - conda-forge/noarch::pytest==8.1.1=pyhd8ed1ab_0
  - conda-forge/noarch::qtpy==2.4.1=pyhd8ed1ab_0
  - conda-forge/linux-64::sip==6.7.12=py310hc6cd4ac_0
  - conda-forge/noarch::flask==3.0.2=pyhd8ed1ab_0
  - conda-forge/noarch::importlib_metadata==7.1.0=hd8ed1ab_0
  - conda-forge/linux-64::pyqt5-sip==12.12.2=py310hc6cd4ac_5
  - conda-forge/noarch::pytoolconfig==1.2.5=pyhd8ed1ab_0
  - conda-forge/noarch::qdarkstyle==3.1=pyhd8ed1ab_0
  - conda-forge/noarch::qtawesome==1.3.0=pyh9208f05_0
  - conda-forge/noarch::yapf==0.40.1=pyhd8ed1ab_0
  - conda-forge/noarch::dask-core==2024.3.1=pyhd8ed1ab_0


In [7]:
import pandas as pd
import os
import pickle
import numpy as np
import sklearn
from sklearn.metrics import r2_score
import boto3
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.tuner import (
    IntegerParameter,
    ContinuousParameter,
    HyperparameterTuner
)
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from feature_engine.selection import SelectBySingleFeaturePerformance


from sklearn.preprocessing import(
    OneHotEncoder,
    MinMaxScaler,
    PowerTransformer,
    FunctionTransformer,
    StandardScaler,
    OrdinalEncoder
    
)
from feature_engine.outliers import Winsorizer
from feature_engine.encoding import(
    RareLabelEncoder,
    MeanEncoder,
    CountFrequencyEncoder,

)

from feature_engine.datetime import DatetimeFeatures
import warnings
import matplotlib.pyplot as plt


In [8]:
import xgboost as xgb

# display settings

In [14]:
pd.set_option("display.max_columns",None)

In [15]:
sklearn.set_config(transform_output="pandas")

In [16]:
warnings.filterwarnings("ignore")

# read datasets

In [17]:
train = pd.read_csv("train.csv")
train

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-06-12,Kolkata,Banglore,16:30:00,12:00:00,1170,1.0,In-flight meal not included,8529
1,Jet Airways,2019-05-18,Banglore,Delhi,11:10:00,14:05:00,175,0.0,In-flight meal not included,5198
2,Multiple Carriers,2019-06-12,Delhi,Cochin,08:45:00,19:00:00,615,1.0,No Info,11789
3,Multiple Carriers,2019-04-24,Delhi,Cochin,07:10:00,16:10:00,540,1.0,In-flight meal not included,6093
4,Jet Airways,2019-03-27,Delhi,Cochin,15:05:00,04:25:00,800,1.0,No Info,12242
...,...,...,...,...,...,...,...,...,...,...
1704,Jet Airways,2019-06-03,Delhi,Cochin,18:15:00,19:00:00,1485,1.0,In-flight meal not included,10262
1705,Multiple Carriers,2019-05-09,Delhi,Cochin,07:30:00,19:15:00,705,1.0,No Info,8266
1706,Jet Airways,2019-06-09,Delhi,Cochin,20:55:00,04:25:00,450,1.0,In-flight meal not included,10577
1707,Jet Airways,2019-06-12,Kolkata,Banglore,06:30:00,16:20:00,590,1.0,In-flight meal not included,9899


In [18]:
val = pd.read_csv("val.csv")
val

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-06-12,Kolkata,Banglore,16:30:00,12:00:00,1170,1.0,In-flight meal not included,8529
1,Jet Airways,2019-05-18,Banglore,Delhi,11:10:00,14:05:00,175,0.0,In-flight meal not included,5198
2,Multiple Carriers,2019-06-12,Delhi,Cochin,08:45:00,19:00:00,615,1.0,No Info,11789
3,Multiple Carriers,2019-04-24,Delhi,Cochin,07:10:00,16:10:00,540,1.0,In-flight meal not included,6093
4,Jet Airways,2019-03-27,Delhi,Cochin,15:05:00,04:25:00,800,1.0,No Info,12242
...,...,...,...,...,...,...,...,...,...,...
1704,Jet Airways,2019-06-03,Delhi,Cochin,18:15:00,19:00:00,1485,1.0,In-flight meal not included,10262
1705,Multiple Carriers,2019-05-09,Delhi,Cochin,07:30:00,19:15:00,705,1.0,No Info,8266
1706,Jet Airways,2019-06-09,Delhi,Cochin,20:55:00,04:25:00,450,1.0,In-flight meal not included,10577
1707,Jet Airways,2019-06-12,Kolkata,Banglore,06:30:00,16:20:00,590,1.0,In-flight meal not included,9899


In [19]:
test = pd.read_csv("test.csv")
test

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Multiple Carriers,2019-05-21,Delhi,Cochin,15:05:00,01:30:00,625,2.0,No Info,16655
1,Goair,2019-06-03,Delhi,Cochin,10:35:00,19:35:00,540,1.0,No Info,4959
2,Vistara,2019-05-09,Kolkata,Banglore,20:20:00,09:05:00,765,1.0,No Info,9187
3,Indigo,2019-05-24,Chennai,Kolkata,14:45:00,17:05:00,140,0.0,No Info,3858
4,Jet Airways,2019-05-21,Delhi,Cochin,22:50:00,04:25:00,335,1.0,In-flight meal not included,12898
...,...,...,...,...,...,...,...,...,...,...
2132,Multiple Carriers,2019-06-27,Delhi,Cochin,08:00:00,21:00:00,780,1.0,No Info,7408
2133,Spicejet,2019-03-06,Kolkata,Banglore,17:15:00,19:45:00,150,0.0,No Info,4622
2134,Air Asia,2019-04-21,Delhi,Cochin,07:55:00,22:25:00,870,1.0,No Info,7452
2135,Indigo,2019-03-09,Delhi,Cochin,02:00:00,07:45:00,345,1.0,No Info,7060


# Data preprocessing

In [22]:
# airline
air_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("grouper", RareLabelEncoder(tol=0.1, replace_with="Other", n_categories=2)),
    ("encoder", OneHotEncoder(sparse_output=False, handle_unknown="ignore"))
])

#doj
feature_to_extract = ["month", "week", "day_of_week", "day_of_year"]

doj_transformer = Pipeline(steps=[
    ("dt", DatetimeFeatures(features_to_extract=feature_to_extract, yearfirst=True, format="mixed")),
    ("scaler", MinMaxScaler())
])

# source & destination
location_pipe1 = Pipeline(steps=[
    ("grouper", RareLabelEncoder(tol=0.1, replace_with="Other", n_categories=2)),
    ("encoder", MeanEncoder()),
    ("scaler", PowerTransformer())
])

def is_north(X):
    columns = X.columns.to_list()
    north_cities = ["Delhi", "Kolkata", "Mumbai", "New Delhi"]
    return (
        X
        .assign(**{
            f"{col}_is_north": X.loc[:, col].isin(north_cities).astype(int)
            for col in columns
        })
        .drop(columns=columns)
    )

location_transformer = FeatureUnion(transformer_list=[
    ("part1", location_pipe1),
    ("part2", FunctionTransformer(func=is_north))
])

# dep_time & arrival_time
time_pipe1 = Pipeline(steps=[
    ("dt", DatetimeFeatures(features_to_extract=["hour", "minute"])),
    ("scaler", MinMaxScaler())
])

def part_of_day(X, morning=4, noon=12, eve=16, night=20):
    columns = X.columns.to_list()
    X_temp = X.assign(**{
        col: pd.to_datetime(X.loc[:, col]).dt.hour
        for col in columns
    })

    return (
        X_temp
        .assign(**{
            f"{col}_part_of_day": np.select(
                [X_temp.loc[:, col].between(morning, noon, inclusive="left"),
                 X_temp.loc[:, col].between(noon, eve, inclusive="left"),
                 X_temp.loc[:, col].between(eve, night, inclusive="left")],
                ["morning", "afternoon", "evening"],
                default="night"
            )
            for col in columns
        })
        .drop(columns=columns)
    )

time_pipe2 = Pipeline(steps=[
    ("part", FunctionTransformer(func=part_of_day)),
    ("encoder", CountFrequencyEncoder()),
    ("scaler", MinMaxScaler())
])

time_transformer = FeatureUnion(transformer_list=[
    ("part1", time_pipe1),
    ("part2", time_pipe2)
])

# duration
class RBFPercentileSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, variables=None, percentiles=[0.25, 0.5, 0.75], gamma=0.1):
        self.variables = variables
        self.percentiles = percentiles
        self.gamma = gamma


    def fit(self, X, y=None):
        if not self.variables:
            self.variables = X.select_dtypes(include="number").columns.to_list()

        self.reference_values_ = {
            col: (
                X
                .loc[:, col]
                .quantile(self.percentiles)
                .values
                .reshape(-1, 1)
            )
            for col in self.variables
        }

        return self


    def transform(self, X):
        objects = []
        for col in self.variables:
            columns = [f"{col}_rbf_{int(percentile * 100)}" for percentile in self.percentiles]
            obj = pd.DataFrame(
                data=rbf_kernel(X.loc[:, [col]], Y=self.reference_values_[col], gamma=self.gamma),
                columns=columns
            )
            objects.append(obj)
        return pd.concat(objects, axis=1)
    

def duration_category(X, short=180, med=400):
    return (
        X
        .assign(duration_cat=np.select([X.duration.lt(short),
                                    X.duration.between(short, med, inclusive="left")],
                                    ["short", "medium"],
                                    default="long"))
        .drop(columns="duration")
    )

def is_over(X, value=1000):
    return (
        X
        .assign(**{
            f"duration_over_{value}": X.duration.ge(value).astype(int)
        })
        .drop(columns="duration")
    )

duration_pipe1 = Pipeline(steps=[
    ("rbf", RBFPercentileSimilarity()),
    ("scaler", PowerTransformer())
])

duration_pipe2 = Pipeline(steps=[
    ("cat", FunctionTransformer(func=duration_category)),
    ("encoder", OrdinalEncoder(categories=[["short", "medium", "long"]]))
])

duration_union = FeatureUnion(transformer_list=[
    ("part1", duration_pipe1),
    ("part2", duration_pipe2),
    ("part3", FunctionTransformer(func=is_over)),
    ("part4", StandardScaler())
])

duration_transformer = Pipeline(steps=[
    ("outliers", Winsorizer(capping_method="iqr", fold=1.5)),
    ("imputer", SimpleImputer(strategy="median")),
    ("union", duration_union)
])

# total_stops
def is_direct(X):
    return X.assign(is_direct_flight=X.total_stops.eq(0).astype(int))


total_stops_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("", FunctionTransformer(func=is_direct))
])

# additional_info
info_pipe1 = Pipeline(steps=[
    ("group", RareLabelEncoder(tol=0.1, n_categories=2, replace_with="Other")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

def have_info(X):
    return X.assign(additional_info=X.additional_info.ne("No Info").astype(int))

info_union = FeatureUnion(transformer_list=[
("part1", info_pipe1),
("part2", FunctionTransformer(func=have_info))
])

info_transformer = Pipeline(steps=[
("imputer", SimpleImputer(strategy="constant", fill_value="unknown")),
("union", info_union)
])

# column transformer
column_transformer = ColumnTransformer(transformers=[
("air", air_transformer, ["airline"]),
("doj", doj_transformer, ["date_of_journey"]),
("location", location_transformer, ["source", 'destination']),
("time", time_transformer, ["dep_time", "arrival_time"]),
("dur", duration_transformer, ["duration"]),
("stops", total_stops_transformer, ["total_stops"]),
("info", info_transformer, ["additional_info"])
], remainder="passthrough")

# feature selector
estimator = RandomForestRegressor(n_estimators=10, max_depth=3, random_state=42)

selector = SelectBySingleFeaturePerformance(
estimator=estimator,
scoring="r2",
threshold=0.1
) 

# preprocessor
preprocessor = Pipeline(steps=[
("ct", column_transformer),
("selector", selector)
])

In [23]:
preprocessor.fit(
    train.drop(columns="price"),
    train.price.copy()
)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('air',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('grouper',
                                                                   RareLabelEncoder(n_categories=2,
                                                                                    replace_with='Other',
                                                                                    tol=0.1)),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['airline']),
                                                 ('doj',
                                                  Pipeline(steps=[('dt',
                                                                   Datetime...
                                                                                                                                     replace_with='Other',
                                                                                                                                     tol=0.1)),
                                                                                                                   ('encoder',
                                                                                                                    OneHotEncoder(handle_unknown='ignore',
                                                                                                                                  sparse_output=False))])),
                                                                                                  ('part2',
                                                                                                   FunctionTransformer(func=<function have_info at 0x7fdacdcfbc70>))]))]),
                                                  ['additional_info'])])),
                ('selector',
                 SelectBySingleFeaturePerformance(estimator=RandomForestRegressor(max_depth=3,
                                                                                  n_estimators=10,
                                                                                  random_state=42),
                                                  scoring='r2',
                                                  threshold=0.1))])

In [24]:
preprocessor.transform(train.drop(columns="price"))

,air__airline_Indigo,air__airline_Jet Airways,air__airline_Other,doj__date_of_journey_week,doj__date_of_journey_day_of_year,location__source,location__destination,dur__duration_cat,dur__duration_over_1000,dur__duration,stops__total_stops,stops__is_direct_flight
0,0.0,1.0,0.0,0.882353,0.872881,-0.186011,-0.152980,2.0,1,1.011302,1.0,0
1,0.0,1.0,0.0,0.647059,0.661017,-0.999315,-1.812625,0.0,0,-0.916232,0.0,1
2,0.0,0.0,0.0,0.882353,0.872881,0.990136,0.992114,2.0,0,-0.063855,1.0,0
3,0.0,0.0,0.0,0.470588,0.457627,0.990136,0.992114,2.0,0,-0.209147,1.0,0
4,0.0,1.0,0.0,0.235294,0.220339,0.990136,0.992114,2.0,0,0.294531,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1704,0.0,1.0,0.0,0.823529,0.796610,0.990136,0.992114,2.0,1,1.621526,1.0,0
1705,0.0,0.0,0.0,0.588235,0.584746,0.990136,0.992114,2.0,0,0.110495,1.0,0
1706,0.0,1.0,0.0,0.823529,0.847458,0.990136,0.992114,2.0,0,-0.383496,1.0,0
1707,0.0,1.0,0.0,0.882353,0.872881,-0.186011,-0.152980,2.0,0,-0.112286,1.0,0


# preprocess the data upload to S3 bucket

In [44]:
BUCKET_NAME = "sagemaker-flights-prediction-bucket"

DATA_PREFIX = "data"

In [17]:
def get_file_name(name):
    return f"{name}-pre.csv"

In [46]:
def export_data(data, name, pre):
    # split data into X and y subsets
    X = data.drop(columns="price")
    y = data.price.copy()
    
    # transformation
    X_pre = pre.transform(X)
    
    # exporting
    file_name = get_file_name(name)
    (
        y
        .to_frame()
        .join(X_pre)
        .to_csv(file_name, index=False, header=False)
    )

In [47]:
def upload_to_bucket(name):
    file_name = get_file_name(name)
    
    (
        boto3
        .Session()
        .resource("s3")
        .Bucket(BUCKET_NAME)
        .Object(os.path.join(DATA_PREFIX, f"{name}/{name}.csv"))
        .upload_file(file_name)
    )


In [48]:
def export_and_upload_bucket(data, name, pre):
    export_data(data, name, pre)
    upload_to_bucket(name)

In [49]:
export_and_upload_bucket(train, "train", preprocessor)


In [50]:
export_and_upload_bucket(val, "val", preprocessor)


In [51]:
export_and_upload_bucket(test, "test", preprocessor)


# Model HyperparameterTuning setup 

In [53]:
session = sagemaker.Session()
region_name = session.boto_region_name

In [54]:
output_path = f"s3://{BUCKET_NAME}/model/output"


In [55]:
model = Estimator(
    image_uri=sagemaker.image_uris.retrieve("xgboost", region_name, "1.2-1"),
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m4.xlarge",
    volume_size=5,
    output_path=output_path,
    use_spot_instances=True,
    max_run=300,
    max_wait=600,
    sagemaker_session=session
)

In [56]:
model.set_hyperparameters(
    objective="reg:linear",
    num_round=10,
    eta=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    alpha=0.1
)

In [57]:
hyperparameter_ranges = {
    "eta": ContinuousParameter(0.05, 0.2),
    "alpha": ContinuousParameter(0, 1),
    "max_depth": IntegerParameter(3, 5)
}

In [58]:
tuner = HyperparameterTuner(
    estimator=model,
    objective_metric_name="validation:rmse",
    hyperparameter_ranges=hyperparameter_ranges,
    strategy="Bayesian",
    objective_type="Minimize"
)

# Data channels

In [59]:
def get_data_channel(name):
    bucket_path = f"s3://{BUCKET_NAME}/{DATA_PREFIX}/{name}"
    return TrainingInput(bucket_path, content_type="csv")

In [60]:
train_data_channel = get_data_channel("train")
train_data_channel

In [61]:
val_data_channel = get_data_channel("val")


In [62]:
data_channels = {
    "train": train_data_channel,
    "validation": val_data_channel
}


# train  model

In [63]:
tuner.fit(data_channels)


No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


......................................!


# deploying model on sagemaker but we will deploy it on streamlit

In [ ]:
#tuner.best_estimator().deploy()

# model evaluation

In [5]:
with open("xgboost-model", "rb") as f:
    best_model = pickle.load(f)
    
best_model

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [10:16:20] WARNING: /tmp/pip-install-yajope65/xgboost_d83b35533b0b4e50bc061962e0f591c2/cpp_src/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [10:16:20] WARNING: /tmp/pip-install-yajope65/xgboost_d83b35533b0b4e50bc061962e0f591c2/cpp_src/src/collective/../data/../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, Us

In [14]:
def evaluate_model(name):
    file_name = get_file_name(name)
    data = pd.read_csv(file_name)
    
    X = xgb.DMatrix(data.iloc[:, 1:])
    y = data.iloc[:, 0].copy()
    
    pred = best_model.predict(X)
    
    return r2_score(y, pred)

In [6]:
pd.read_csv("train-pre.csv")

,8529,0.0,1.0,0.0.1,0.8823529411764705,0.8728813559322035,-0.18601095667460515,-0.15297950872701432,2.0,1,1.0113018238442264,1.0.1,0
0,5198,0.0,1.0,0.0,0.647059,0.661017,-0.999315,-1.812625,0.0,0,-0.916232,0.0,1
1,11789,0.0,0.0,0.0,0.882353,0.872881,0.990136,0.992114,2.0,0,-0.063855,1.0,0
2,6093,0.0,0.0,0.0,0.470588,0.457627,0.990136,0.992114,2.0,0,-0.209147,1.0,0
3,12242,0.0,1.0,0.0,0.235294,0.220339,0.990136,0.992114,2.0,0,0.294531,1.0,0
4,9345,0.0,0.0,1.0,0.705882,0.686441,-0.186011,-0.152980,1.0,0,-0.509416,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1703,10262,0.0,1.0,0.0,0.823529,0.796610,0.990136,0.992114,2.0,1,1.621526,1.0,0
1704,8266,0.0,0.0,0.0,0.588235,0.584746,0.990136,0.992114,2.0,0,0.110495,1.0,0
1705,10577,0.0,1.0,0.0,0.823529,0.847458,0.990136,0.992114,2.0,0,-0.383496,1.0,0
1706,9899,0.0,1.0,0.0,0.882353,0.872881,-0.186011,-0.152980,2.0,0,-0.112286,1.0,0


In [15]:
def evaluate_model(name):
    file_name = get_file_name(name)
    data = pd.read_csv(file_name)
    
    X = xgb.DMatrix(data.iloc[:, 1:])
    y = data.iloc[:, 0].copy()
    
    pred = best_model.predict(X)
    
    return r2_score(y, pred)

In [18]:
evaluate_model("train")


0.6524790414358335

In [19]:
evaluate_model("val")


0.6524790414358335